In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.float_format', '{:.2f}'.format)
np.set_printoptions(formatter={'float_kind': "{:.2f}".format})
%config InlineBackend.figure_format = 'svg'

In [2]:
path = '/home/arthur/Project/GeekBrains/DB_research/!ADDS/hw_1/orders_20190822.csv'
df = pd.read_csv(path, sep=';')
df

,id_o,user_id,price,o_date
0,1234491,337544,"539,000",01.01.2016
1,1234494,171642,"153,300",01.01.2016
2,1234497,260596,"55,300",01.01.2016
3,1234498,1105609,"752,500",01.01.2016
4,1234500,982696,"4410,000",01.01.2016
...,...,...,...,...
2002799,6945524,5806002,"825,300",31.12.2017
2002800,6945526,5919142,"4934,300",31.12.2017
2002801,6945527,1574166,"804,300",31.12.2017
2002802,6945528,5919156,"5019,700",31.12.2017


In [3]:
# задание соответствующих форматов столбцам
df['o_date'] = pd.to_datetime(df['o_date'], format='%d.%m.%Y')
df['year'] = df['o_date'].dt.year
df['price'] = df['price'].str.replace(',', '.').astype('float')
df['month'] = df['o_date'].dt.month

# Главная задача: сделать RFM-анализ на основе данных по продажам за 2 года (из предыдущего дз). Что делаем:

## 1. Определяем критерии для каждой буквы R, F, M (т.е. к примеру, R – 3 для клиентов, которые покупали <= 30 дней от последней даты в базе, R – 2 для клиентов, которые покупали > 30 и менее 60 дней от последней даты в базе и т.д.)

### Критерии
- R (2018-01-01 - maxdate):
    - 1: 61+ дней
    - 2: 31-60 дней
    - 3: 0-30 дней
- F:
    - 1: 1-2
    - 2: 3-5
    - 3: 6+
- M:
    - 1: 0-10
    - 2: 11-20
    - 3: 21+

In [4]:
df.head()

,id_o,user_id,price,o_date,year,month
0,1234491,337544,539.00,2016-01-01,2016,1
1,1234494,171642,153.30,2016-01-01,2016,1
2,1234497,260596,55.30,2016-01-01,2016,1
3,1234498,1105609,752.50,2016-01-01,2016,1
4,1234500,982696,4410.00,2016-01-01,2016,1


In [5]:
df_pivot = df.pivot_table(index=['user_id'], values=['o_date', 'price'], 
                           aggfunc={'o_date': 'max', 'price': ['count', 'sum']})
df_pivot.columns = ['max_o_date', 'o_count', 'o_sum']
df_pivot['date_delta'] = np.datetime64('2018-01-01') - df_pivot['max_o_date']
df_pivot

,max_o_date,o_count,o_sum,date_delta
user_id,,,,
0,2016-12-08,1,494.20,389 days
1,2017-08-01,2,13844.60,153 days
76,2017-11-09,2,1863.40,53 days
90,2017-12-20,4,6561.80,12 days
91,2017-10-11,4,5413.10,82 days
...,...,...,...,...
5919114,2017-12-31,1,1272.60,1 days
5919118,2017-12-31,1,7242.20,1 days
5919128,2017-12-31,1,886.90,1 days


## 2. Для каждого пользователя получаем набор из 3 цифр (от 111 до 333, где 333 – самые классные пользователи)


In [6]:
# проставляем R
df_pivot.loc[(df_pivot['date_delta'] <= np.timedelta64('30', 'D')), 'R'] = 1
df_pivot.loc[(df_pivot['date_delta'] >= np.timedelta64('31', 'D')) & 
             (df_pivot['date_delta'] <= np.timedelta64('60', 'D')), 'R'] = 2
df_pivot.loc[(df_pivot['date_delta'] >= np.timedelta64('61', 'D')), 'R'] = 3

# проставляем F
df_pivot.loc[(df_pivot['o_count'] <= 2), 'F'] = 1
df_pivot.loc[(df_pivot['o_count'] > 2) & 
              (df_pivot['o_count'] <= 5), 'F'] = 2
df_pivot.loc[(df_pivot['o_count'] > 5), 'F'] = 3

# проставляем М
df_pivot.loc[(df_pivot['o_sum'] <= 10), 'M'] = 1
df_pivot.loc[(df_pivot['o_sum'] > 1) & 
             (df_pivot['o_sum'] <= 20), 'M'] = 2
df_pivot.loc[(df_pivot['o_sum'] > 20), 'M'] = 3

df_pivot

,max_o_date,o_count,o_sum,date_delta,R,F,M
user_id,,,,,,,
0,2016-12-08,1,494.20,389 days,3.00,1.00,3.00
1,2017-08-01,2,13844.60,153 days,3.00,1.00,3.00
76,2017-11-09,2,1863.40,53 days,2.00,1.00,3.00
90,2017-12-20,4,6561.80,12 days,1.00,2.00,3.00
91,2017-10-11,4,5413.10,82 days,3.00,2.00,3.00
...,...,...,...,...,...,...,...
5919114,2017-12-31,1,1272.60,1 days,1.00,1.00,3.00
5919118,2017-12-31,1,7242.20,1 days,1.00,1.00,3.00
5919128,2017-12-31,1,886.90,1 days,1.00,1.00,3.00


## 3. Вводим группировку, к примеру, 333 и 233 – это Vip, 1XX – это Lost, остальные Regular ( можете ввести боле глубокую сегментацию)


In [7]:
df_pivot['marker'] = df_pivot['R'].astype('int64').astype('str').str.cat(
    df_pivot['F'].astype('int64').astype('str')).str.cat(
    df_pivot['M'].astype('int64').astype('str')).astype('int64')
df_pivot

,max_o_date,o_count,o_sum,date_delta,R,F,M,marker
user_id,,,,,,,,
0,2016-12-08,1,494.20,389 days,3.00,1.00,3.00,313
1,2017-08-01,2,13844.60,153 days,3.00,1.00,3.00,313
76,2017-11-09,2,1863.40,53 days,2.00,1.00,3.00,213
90,2017-12-20,4,6561.80,12 days,1.00,2.00,3.00,123
91,2017-10-11,4,5413.10,82 days,3.00,2.00,3.00,323
...,...,...,...,...,...,...,...,...
5919114,2017-12-31,1,1272.60,1 days,1.00,1.00,3.00,113
5919118,2017-12-31,1,7242.20,1 days,1.00,1.00,3.00,113
5919128,2017-12-31,1,886.90,1 days,1.00,1.00,3.00,113


In [8]:
df_pivot.loc[(df_pivot['marker'] == 333), 'category'] = 'vip'
df_pivot.loc[(df_pivot['marker'] < 333) & (df_pivot['marker'] >= 133), 'category'] = 'regular'
df_pivot.loc[(df_pivot['marker'] < 133), 'category'] = 'lost'

df_pivot.pivot_table(index='category', values='marker', aggfunc='count')

,marker
category,
lost,118957
regular,867594
vip,28568


## 4. Для каждой группы из п. 3 находим кол-во пользователей, кот. попали в них и % товарооборота, которое они сделали на эти 2 года.


In [9]:
df_pivot['o_share'] = df_pivot['o_sum'] / df_pivot['o_sum'].sum() * 100

In [10]:
result = df_pivot.pivot_table(index='category', values=['marker', 'o_share'], 
                     aggfunc={'marker': 'count', 'o_share': 'sum'})
result.columns = ['кол-во', 'доля тов-та']
result

,кол-во,доля тов-та
category,,
lost,118957,8.84
regular,867594,76.56
vip,28568,14.60


## 5. Проверяем, что общее кол-во пользователей бьется с суммой кол-во пользователей по группам из п. 3 (если у вас есть логические ошибки в создании групп, у вас не собьются цифры). То же самое делаем и по деньгам.

In [11]:
# сбиваем количество пользователей
result['кол-во'].sum() == df_pivot.shape[0]

True

In [12]:
# сбиваем сумму товарооборота
result['доля тов-та'].sum()

99.99999999999523